**Exemplo de Streaming Para Leitura de Arquivos de Um Diretório**

In [1]:
#cria a seção a ser utiliza para estabelecer a conexão 
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

spark = SparkSession \
    .builder \
    .appName("StructuredNetworkWordCount") \
    .getOrCreate()

In [2]:
#cria o dataframe que será responsável por ler cada uma das linhas recebidas dos arquivos adicionados no diretório
files_dir = spark.readStream\
    .format("text")\
    .option("inferSchema", "true")\
    .option("maxFilesPerTrigger", 1)\
    .load("/home/tulio/Documents/Aulas_PFC/arquivos/*.txt")

In [3]:
#verifica se criou o streaming
files_dir.isStreaming

True

In [4]:
# Divide as linhas recebidas em cada palavra
words = files_dir.select(
   explode(
       split(files_dir.value, " ")
   ).alias("word")
)

# cria o novo dataframe a ser responsável por contar a quantidade de palavras digitadas
wordCounts = words.groupBy("word").count()


In [5]:
#ordena as palavras que mais aparecem
from pyspark.sql.functions import desc
wordCounts = wordCounts.sort(desc("count"))

In [ ]:
# Define a consulta (query) e como deve ser realizada a saída (sink) para o stream criado 
query = wordCounts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start() #inicia a "query"


query.awaitTermination() #aguarda até que a "streaming query" termine 